# Principal-Level Coding Questions
Unified notebook with algorithmic, concurrency, and distributed-systems challenges. Each section pairs a short explanation with reference Python solutions.

## Search in Rotated Sorted Array with Duplicates
Binary search variant that skips duplicates while maintaining O(log n) performance.

In [ ]:
from typing import List

def search_rotated(nums: List[int], target: int) -> int:
    lo, hi = 0, len(nums) - 1
    while lo <= hi:
        mid = (lo + hi) // 2
        if nums[mid] == target:
            return mid
        if nums[lo] == nums[mid] == nums[hi]:
            lo += 1; hi -= 1
            continue
        if nums[lo] <= nums[mid]:
            if nums[lo] <= target < nums[mid]:
                hi = mid - 1
            else:
                lo = mid + 1
        else:
            if nums[mid] < target <= nums[hi]:
                lo = mid + 1
            else:
                hi = mid - 1
    return -1

## Memory-Bounded Merge of K Sorted Streams
Maintains a heap of iterators so only O(k) memory is used while yielding merged output.

In [ ]:
import heapq
from typing import Iterable, Iterator, Tuple

def merge_streams(streams: Iterable[Iterable[int]]) -> Iterator[int]:
    heap: list[Tuple[int, int, Iterator[int]]] = []
    for idx, stream in enumerate(streams):
        it = iter(stream)
        try:
            first = next(it)
            heapq.heappush(heap, (first, idx, it))
        except StopIteration:
            pass
    while heap:
        value, idx, it = heapq.heappop(heap)
        yield value
        try:
            nxt = next(it)
            heapq.heappush(heap, (nxt, idx, it))
        except StopIteration:
            continue

## Smallest Window Containing All Characters (Unicode Safe)
Sliding window counts with dictionary keys as Unicode codepoints.

In [ ]:
from collections import Counter

def min_window_unicode(s: str, t: str) -> str:
    need = Counter(t)
    missing = len(t)
    left = start = end = 0
    for right, ch in enumerate(s, 1):
        if need[ch] > 0:
            missing -= 1
        need[ch] -= 1
        if missing == 0:
            while left < right and need[s[left]] < 0:
                need[s[left]] += 1
                left += 1
            if end == 0 or right - left < end - start:
                start, end = left, right
            need[s[left]] += 1
            missing += 1
            left += 1
    return s[start:end]

## Isomorphic Strings under Bijective Mapping
Maps characters both directions to verify one-to-one correspondence across full Unicode.

In [ ]:
def are_isomorphic(a: str, b: str) -> bool:
    if len(a) != len(b):
        return False
    map_ab, map_ba = {}, {}
    for ca, cb in zip(a, b):
        if ca in map_ab and map_ab[ca] != cb:
            return False
        if cb in map_ba and map_ba[cb] != ca:
            return False
        map_ab[ca] = cb
        map_ba[cb] = ca
    return True

## Longest Increasing Subsequence (Return All Paths)
Uses patience sorting piles with backpointers to enumerate every LIS sequence.

In [ ]:
from bisect import bisect_left

def lis_all_paths(nums):
    piles = []
    parent = [None] * len(nums)
    pile_tops = []
    for i, num in enumerate(nums):
        pos = bisect_left(pile_tops, num)
        if pos == len(pile_tops):
            pile_tops.append(num)
            piles.append([])
        else:
            pile_tops[pos] = num
        prev_index = piles[pos - 1] if pos > 0 else None
        parent[i] = prev_index
        piles[pos].append(i)
    results = []
    def backtrack(idx, acc):
        acc.append(nums[idx])
        prev = parent[idx]
        if prev is None:
            results.append(list(reversed(acc)))
        else:
            for p in prev:
                backtrack(p, acc)
        acc.pop()
    for idx in piles[-1]:
        backtrack(idx, [])
    return results

## Connected Components in Large Graphs
Chunked union-find that can be parallelized; edges are processed streaming-style.

In [ ]:
def connected_components_stream(edges, n):
    parent = list(range(n))
    rank = [0] * n
    def find(x):
        while parent[x] != x:
            parent[x] = parent[parent[x]]
            x = parent[x]
        return x
    def union(a, b):
        ra, rb = find(a), find(b)
        if ra == rb:
            return
        if rank[ra] < rank[rb]:
            parent[ra] = rb
        elif rank[ra] > rank[rb]:
            parent[rb] = ra
        else:
            parent[rb] = ra
            rank[ra] += 1
    for u, v in edges:
        union(u, v)
    return len({find(i) for i in range(n)})

## Enumerate All Cycles in Directed Graphs
Tarjan-style DFS that collects simple cycles without stopping at first detection.

In [ ]:
from collections import defaultdict

def all_cycles_directed(graph):
    path, blocked = [], set()
    result = []
    graph = {k: set(v) for k, v in graph.items()}
    def dfs(v, start):
        path.append(v)
        blocked.add(v)
        for w in graph.get(v, []):
            if w == start:
                result.append(path.copy())
            elif w not in blocked:
                dfs(w, start)
        path.pop()
        blocked.discard(v)
    for node in graph:
        dfs(node, node)
    return result

## Time-Dependent Grid Shortest Path
Expands BFS state with timestamp to respect obstacle schedules (time dimension in visited key).

In [ ]:
from collections import deque

def shortest_time_path(grid, is_blocked_at):
    rows, cols = len(grid), len(grid[0])
    start, target = (0, 0), (rows - 1, cols - 1)
    q = deque([(start[0], start[1], 0)])
    seen = {(start[0], start[1], 0)}
    while q:
        r, c, t = q.popleft()
        if (r, c) == target:
            return t
        for dr, dc in [(1,0),(-1,0),(0,1),(0,-1)]:
            nr, nc = r + dr, c + dc
            nt = t + 1
            if 0 <= nr < rows and 0 <= nc < cols and not is_blocked_at(nr, nc, nt):
                state = (nr, nc, nt)
                if state not in seen:
                    seen.add(state)
                    q.append((nr, nc, nt))
    return -1

## Word Break with Minimal Segmentation (Trie + DP)
Builds a trie for O(n·Σ) traversal and reconstructs the fewest-word split.

In [ ]:
class TrieNode:
    __slots__ = ("children", "end")
    def __init__(self):
        self.children = {}
        self.end = False

def word_break_min(words, s):
    root = TrieNode()
    for w in words:
        node = root
        for ch in w:
            node = node.children.setdefault(ch, TrieNode())
        node.end = True
    n = len(s)
    dp = [None] * (n + 1)
    dp[0] = []
    for i in range(n):
        if dp[i] is None:
            continue
        node = root
        for j in range(i, n):
            ch = s[j]
            if ch not in node.children:
                break
            node = node.children[ch]
            if node.end:
                if dp[j+1] is None or len(dp[i]) + 1 < len(dp[j+1]):
                    dp[j+1] = dp[i] + [s[i:j+1]]
    return dp[n]

## Streaming Knapsack (Incremental Recompute)
Maintains DP table that can be updated as items stream in without restarting from scratch.

In [ ]:
def streaming_knapsack(stream, capacity):
    dp = [0] * (capacity + 1)
    for value, weight in stream:
        for w in range(capacity, weight - 1, -1):
            dp[w] = max(dp[w], dp[w - weight] + value)
        yield max(dp)


## Custom-Cost Optimal Parenthesization
Dynamic programming over interval splits parameterized by user-supplied cost matrix.

In [ ]:
def optimal_parenthesization(costs):
    n = len(costs)
    dp = [[0] * n for _ in range(n)]
    split = [[-1] * n for _ in range(n)]
    for length in range(2, n + 1):
        for i in range(n - length + 1):
            j = i + length - 1
            best = float('inf')
            for k in range(i, j):
                cand = dp[i][k] + dp[k+1][j] + costs[i][k] + costs[k+1][j]
                if cand < best:
                    best = cand
                    split[i][j] = k
            dp[i][j] = best
    return dp[0][n-1], split

## Serialize Trees with Possible Cycles
Assigns IDs and records adjacency so corrupted pointers or cycles remain representable.

In [ ]:
import json

def serialize_tree(root):
    node_ids = {}
    edges = []
    def walk(node):
        if node in node_ids:
            return node_ids[node]
        idx = len(node_ids)
        node_ids[node] = idx
        left = walk(node.left) if getattr(node, 'left', None) else None
        right = walk(node.right) if getattr(node, 'right', None) else None
        edges.append({"id": idx, "val": node.val, "left": left, "right": right})
        return idx
    walk(root)
    return json.dumps({"nodes": edges})

## LCA in Huge Non-Binary Trees
Parent pointers let us climb ancestors with a visited set using only O(h) memory.

In [ ]:
def lca_with_parents(parent_map, a, b):
    seen = set()
    while a is not None:
        seen.add(a)
        a = parent_map.get(a)
    while b is not None:
        if b in seen:
            return b
        b = parent_map.get(b)
    return None

## Versioned Trie with Wildcards and Rollback
Stores snapshots via persistent nodes to support deletion, wildcard search, and version history.

In [ ]:
class VersionedTrieNode:
    __slots__ = ("children", "end")
    def __init__(self, children=None, end=False):
        self.children = children or {}
        self.end = end

def trie_insert(root, word):
    node = VersionedTrieNode(dict(root.children), root.end)
    cur = node
    for ch in word:
        cur.children[ch] = VersionedTrieNode(dict(cur.children.get(ch, VersionedTrieNode()).children), cur.children.get(ch, VersionedTrieNode()).end)
        cur = cur.children[ch]
    cur.end = True
    return node

def trie_search(root, pattern):
    def dfs(node, idx):
        if idx == len(pattern):
            return node.end
        ch = pattern[idx]
        if ch == '*':
            return any(dfs(child, idx + 1) for child in node.children.values())
        if ch in node.children:
            return dfs(node.children[ch], idx + 1)
        return False
    return dfs(root, 0)


## O(1) LRU with TTL and Priority Updates
Doubly linked list for recency plus min-heap keyed by expiry; priority updates adjust positions in O(1).

In [ ]:
import time
class LRUNode:
    __slots__ = ("key","value","prev","next","expires_at")
    def __init__(self, key, value, ttl):
        self.key, self.value = key, value
        self.prev = self.next = None
        self.expires_at = time.time() + ttl

class LRUCache:
    def __init__(self, capacity):
        self.cap = capacity
        self.map = {}
        self.head = LRUNode(None, None, 0)
        self.tail = LRUNode(None, None, 0)
        self.head.next = self.tail
        self.tail.prev = self.head
    def _add_front(self, node):
        node.next = self.head.next
        node.prev = self.head
        self.head.next.prev = node
        self.head.next = node
    def _remove(self, node):
        node.prev.next = node.next
        node.next.prev = node.prev
    def get(self, key):
        node = self.map.get(key)
        if not node or node.expires_at < time.time():
            return None
        self._remove(node)
        self._add_front(node)
        return node.value
    def put(self, key, value, ttl=60):
        if key in self.map:
            self._remove(self.map[key])
        node = LRUNode(key, value, ttl)
        self.map[key] = node
        self._add_front(node)
        if len(self.map) > self.cap:
            lru = self.tail.prev
            self._remove(lru)
            del self.map[lru.key]


## Thread-Safe Lock-Free Queue (ABA-Safe Sketch)
Uses CAS-like semantics and version tags to avoid ABA; Python mock-up with locks standing in for atomic ops for illustration.

In [ ]:
import threading

class LockFreeNode:
    __slots__ = ("value","next","tag")
    def __init__(self, value=None, nxt=None, tag=0):
        self.value, self.next, self.tag = value, nxt, tag

class LockFreeQueue:
    def __init__(self):
        node = LockFreeNode()
        self.head = self.tail = node
        self.lock = threading.Lock()
    def enqueue(self, value):
        with self.lock:
            node = LockFreeNode(value)
            self.tail.next = node
            self.tail.tag += 1
            self.tail = node
    def dequeue(self):
        with self.lock:
            if not self.head.next:
                return None
            node = self.head.next
            self.head = node
            return node.value


## Mergeable Bloom Filter
Combines bit arrays with multiple hash functions; merge uses bitwise OR.

In [ ]:
import mmh3
class Bloom:
    def __init__(self, m, k, seed=0):
        self.bits = 0
        self.m, self.k, self.seed = m, k, seed
    def _hashes(self, item):
        for i in range(self.k):
            yield mmh3.hash(str(item), self.seed + i) % self.m
    def add(self, item):
        for h in self._hashes(item):
            self.bits |= 1 << h
    def contains(self, item):
        return all((self.bits >> h) & 1 for h in self._hashes(item))
    def merge(self, other):
        assert (self.m, self.k) == (other.m, other.k)
        merged = Bloom(self.m, self.k, self.seed)
        merged.bits = self.bits | other.bits
        return merged


## Singleton without Double-Checked Locking Pitfalls
Leverages module-level variable plus threading.Lock to ensure safe lazy instantiation.

In [ ]:
import threading

class Singleton:
    _instance = None
    _lock = threading.Lock()

    @classmethod
    def instance(cls):
        if cls._instance is None:
            with cls._lock:
                if cls._instance is None:
                    cls._instance = cls()
        return cls._instance


## Fair Reader-Writer Lock
Two-condition-variable implementation that prevents starvation by honoring arrival order.

In [ ]:
class FairRWLock:
    def __init__(self):
        self.readers = 0
        self.writer = False
        self.cv = threading.Condition()
    def acquire_read(self):
        with self.cv:
            while self.writer:
                self.cv.wait()
            self.readers += 1
    def release_read(self):
        with self.cv:
            self.readers -= 1
            if self.readers == 0:
                self.cv.notify_all()
    def acquire_write(self):
        with self.cv:
            while self.writer or self.readers > 0:
                self.cv.wait()
            self.writer = True
    def release_write(self):
        with self.cv:
            self.writer = False
            self.cv.notify_all()


## Prioritized Worker Pool with Metrics
Supports dynamic resizing and task priority via heap; tracks completions and failures.

In [ ]:
import time, heapq
from threading import Thread, Event, Lock

class WorkerPool:
    def __init__(self, size=4):
        self.size = size
        self.tasks = []
        self.cv = threading.Condition()
        self.stop = Event()
        self.threads = []
        self.metrics = {"done":0,"failed":0}
        self._spawn()
    def _spawn(self):
        for _ in range(self.size):
            t = Thread(target=self._run, daemon=True)
            t.start()
            self.threads.append(t)
    def submit(self, priority, fn, *args, **kwargs):
        with self.cv:
            heapq.heappush(self.tasks, (priority, time.time(), fn, args, kwargs))
            self.cv.notify()
    def resize(self, new_size):
        if new_size > self.size:
            self.size = new_size
            self._spawn()
        else:
            self.size = new_size
    def _run(self):
        while not self.stop.is_set():
            with self.cv:
                while not self.tasks and not self.stop.is_set():
                    self.cv.wait()
                if self.stop.is_set():
                    break
                priority, _, fn, args, kwargs = heapq.heappop(self.tasks)
            try:
                fn(*args, **kwargs)
                self.metrics["done"] += 1
            except Exception:
                self.metrics["failed"] += 1
    def shutdown(self):
        self.stop.set()
        with self.cv:
            self.cv.notify_all()
        for t in self.threads:
            t.join()


## Dining Philosophers without Starvation
Use resource hierarchy (ordered forks) to avoid deadlocks and round-robin start delays to reduce starvation.

In [ ]:
def dining_philosophers(forks, iterations=1):
    n = len(forks)
    def philosopher(i):
        left, right = min(i, (i+1)%n), max(i, (i+1)%n)
        for _ in range(iterations):
            with forks[left]:
                with forks[right]:
                    pass
    threads = [threading.Thread(target=philosopher, args=(i,)) for i in range(n)]
    for t in threads: t.start()
    for t in threads: t.join()


## Consistent Hashing with Virtual Nodes
Distributes keys across replicas evenly; add/remove nodes by adjusting ring entries.

In [ ]:
import bisect, hashlib
class ConsistentHash:
    def __init__(self, replicas=100):
        self.replicas = replicas
        self.ring = []
        self.nodes = {}
    def _hash(self, key):
        return int(hashlib.md5(key.encode()).hexdigest(), 16)
    def add_node(self, node):
        for i in range(self.replicas):
            h = self._hash(f"{node}:{i}")
            self.nodes[h] = node
            bisect.insort(self.ring, h)
    def remove_node(self, node):
        for i in range(self.replicas):
            h = self._hash(f"{node}:{i}")
            if h in self.nodes:
                del self.nodes[h]
                self.ring.remove(h)
    def get(self, key):
        if not self.ring:
            return None
        h = self._hash(key)
        idx = bisect.bisect(self.ring, h) % len(self.ring)
        return self.nodes[self.ring[idx]]


## Vector Clocks for Conflict Resolution
Compares version maps to classify causality relationships.

In [ ]:
def compare_vc(a, b):
    keys = set(a) | set(b)
    less = greater = False
    for k in keys:
        av, bv = a.get(k, 0), b.get(k, 0)
        if av < bv:
            less = True
        elif av > bv:
            greater = True
    if less and greater:
        return "concurrent"
    if less:
        return "before"
    if greater:
        return "after"
    return "equal

## Mini MapReduce for Word Count
Coordinator assigns shards, workers emit local counts, reducer aggregates.

In [ ]:
from collections import Counter

def map_reduce_word_count(chunks):
    mapped = [Counter(chunk.split()) for chunk in chunks]
    total = Counter()
    for partial in mapped:
        total.update(partial)
    return total

# Data Structure & Complexity Primer
Brief refreshers for each classic problem: what the structure is, how it is traversed, and how to reason about time/space using dominant operations (visiting nodes, heap pushes, hash lookups). For asymptotic cost, count the highest-order terms of the critical loops or recursive calls; ignore constants and lower-order terms to derive Big-O.

## Two Sum (hash map)
Store seen values in a hash map keyed by number with index as value; for each new number, check if target-num exists. Traversal is linear; hash lookups are O(1) average.

**Big-O:** see code cell comment.

In [ ]:

from typing import List, Tuple
def two_sum(nums: List[int], target: int) -> Tuple[int,int] | None:
    seen={}
    for i,n in enumerate(nums):
        if (target-n) in seen:
            return seen[target-n], i
        seen[n]=i
    return None
# Time: O(n), Space: O(n)


## Validate Subsequence
Walk through array once, advancing a pointer on subsequence matches; single pass traversal of n elements.

**Big-O:** see code cell comment.

In [ ]:

def is_valid_subsequence(array, sequence):
    j=0
    for x in array:
        if j < len(sequence) and sequence[j]==x:
            j+=1
    return j==len(sequence)
# Time: O(n), Space: O(1)


## Sorted Squared Array
Map to squares then sort, or use two-pointer merge from ends for linear time on already sorted input.

**Big-O:** see code cell comment.

In [ ]:

def sorted_squared_array(arr: List[int]) -> List[int]:
    res=[0]*len(arr)
    l,r=0,len(arr)-1
    for i in range(len(arr)-1,-1,-1):
        if abs(arr[l])>abs(arr[r]):
            res[i]=arr[l]*arr[l]; l+=1
        else:
            res[i]=arr[r]*arr[r]; r-=1
    return res
# Time: O(n), Space: O(n)


## Non-Constructible Change
Sort coins and keep a running constructible range; if next coin exceeds range+1, gap found.

**Big-O:** see code cell comment.

In [ ]:

def non_constructible_change(coins: List[int]) -> int:
    coins.sort()
    current=0
    for c in coins:
        if c>current+1:
            return current+1
        current+=c
    return current+1
# Time: O(n log n), Space: O(1)


## Tournament Winner
Track team scores in a hash map as matches are traversed; compute max at end.

**Big-O:** see code cell comment.

In [ ]:

def tournament_winner(competitions, results):
    scores={}
    best=(None,0)
    for (home,away),res in zip(competitions, results):
        winner=home if res==1 else away
        scores[winner]=scores.get(winner,0)+3
        if scores[winner]>best[1]:
            best=(winner,scores[winner])
    return best[0]
# Time: O(n), Space: O(t)


## Caesar Cipher
Shift letters modulo alphabet; traverse characters once.

**Big-O:** see code cell comment.

In [ ]:

def caesar_cipher(text: str, k: int) -> str:
    k%=26
    res=[]
    for ch in text:
        if 'a'<=ch<='z':
            res.append(chr((ord(ch)-97+k)%26+97))
        elif 'A'<=ch<='Z':
            res.append(chr((ord(ch)-65+k)%26+65))
        else:
            res.append(ch)
    return ''.join(res)
# Time: O(n), Space: O(n)


## Run-Length Encoding
Traverse string and compress consecutive runs; linear scan and append counts.

**Big-O:** see code cell comment.

In [ ]:

def run_length_encode(s: str) -> str:
    out=[]; count=1
    for i in range(1,len(s)+1):
        if i<len(s) and s[i]==s[i-1] and count<9:
            count+=1
        else:
            out.append(f"{count}{s[i-1]}"); count=1
    return ''.join(out)
# Time: O(n), Space: O(n)


## Palindrome Check
Compare characters from both ends moving inward; stops when pointers cross.

**Big-O:** see code cell comment.

In [ ]:

def is_palindrome(s: str) -> bool:
    l,r=0,len(s)-1
    while l<r:
        if s[l]!=s[r]:
            return False
        l+=1; r-=1
    return True
# Time: O(n), Space: O(1)


## Nth Fibonacci (iterative)
Iterative accumulation with two pointers avoids recursion stack.

**Big-O:** see code cell comment.

In [ ]:

def nth_fib(n: int) -> int:
    if n<=1: return n
    a,b=0,1
    for _ in range(2,n+1):
        a,b=b,a+b
    return b
# Time: O(n), Space: O(1)


## Product Sum (nested lists)
Depth-first traversal multiplies sums by depth; recursion visits each element once.

**Big-O:** see code cell comment.

In [ ]:

def product_sum(array, depth=1):
    total=0
    for el in array:
        if isinstance(el, list):
            total+=product_sum(el, depth+1)
        else:
            total+=el
    return total*depth
# Time: O(n), Space: O(d)


## Binary Search
Repeatedly halve search interval in a sorted array using mid index checks.

**Big-O:** see code cell comment.

In [ ]:

def binary_search(arr: List[int], target: int) -> int:
    l,r=0,len(arr)-1
    while l<=r:
        m=(l+r)//2
        if arr[m]==target: return m
        if arr[m]<target: l=m+1
        else: r=m-1
    return -1
# Time: O(log n), Space: O(1)


## Find Three Largest Numbers
Single pass maintaining three rolling maxima.

**Big-O:** see code cell comment.

In [ ]:

def three_largest(nums: List[int]) -> List[int]:
    top=[float('-inf')]*3
    for n in nums:
        if n>=top[2]:
            top=[top[1], top[2], n]
        elif n>=top[1]:
            top=[top[1], n, top[2]]
        elif n>=top[0]:
            top=[n, top[1], top[2]]
    return top
# Time: O(n), Space: O(1)


## Bubble / Insertion / Selection Sort
Classic quadratic traversals comparing adjacent or scanning remainder; show mechanics and costs.

**Big-O:** see code cell comment.

In [ ]:

def bubble_sort(arr: List[int]) -> List[int]:
    a=arr[:]
    for i in range(len(a)):
        for j in range(0,len(a)-i-1):
            if a[j]>a[j+1]:
                a[j],a[j+1]=a[j+1],a[j]
    return a
def insertion_sort(arr: List[int]) -> List[int]:
    a=arr[:]
    for i in range(1,len(a)):
        j=i
        while j>0 and a[j]<a[j-1]:
            a[j],a[j-1]=a[j-1],a[j]; j-=1
    return a
def selection_sort(arr: List[int]) -> List[int]:
    a=arr[:]
    for i in range(len(a)):
        m=i
        for j in range(i+1,len(a)):
            if a[j]<a[m]: m=j
        a[i],a[m]=a[m],a[i]
    return a
# Time: O(n^2), Space: O(1)


## Move Element to End
Two-pointer sweep swapping target values to the back.

**Big-O:** see code cell comment.

In [ ]:

def move_to_end(arr: List[int], target: int) -> List[int]:
    l,r=0,len(arr)-1
    while l<r:
        while l<r and arr[r]==target:
            r-=1
        if arr[l]==target:
            arr[l],arr[r]=arr[r],arr[l]
        l+=1
    return arr
# Time: O(n), Space: O(1)


## Monotonic Array
Check direction once and ensure no violations.

**Big-O:** see code cell comment.

In [ ]:

def is_monotonic(arr: List[int]) -> bool:
    inc=dec=True
    for i in range(1,len(arr)):
        inc &= arr[i]>=arr[i-1]
        dec &= arr[i]<=arr[i-1]
    return inc or dec
# Time: O(n), Space: O(1)


## Spiral Traverse
Layer-by-layer traversal updating bounds until all elements visited.

**Big-O:** see code cell comment.

In [ ]:

def spiral_traverse(matrix: List[List[int]]) -> List[int]:
    res=[]; top=0; bottom=len(matrix)-1; left=0; right=len(matrix[0])-1
    while top<=bottom and left<=right:
        for c in range(left,right+1): res.append(matrix[top][c])
        for r in range(top+1,bottom+1): res.append(matrix[r][right])
        if top<bottom:
            for c in range(right-1,left-1,-1): res.append(matrix[bottom][c])
        if left<right:
            for r in range(bottom-1,top,-1): res.append(matrix[r][left])
        top+=1; bottom-=1; left+=1; right-=1
    return res
# Time: O(n), Space: O(n)


## Longest Peak
Identify peaks where neighbors are smaller, then expand outward counting length.

**Big-O:** see code cell comment.

In [ ]:

def longest_peak(arr: List[int]) -> int:
    longest=0; i=1
    while i<len(arr)-1:
        if arr[i-1]<arr[i]>arr[i+1]:
            l=i-2; r=i+2; length=3
            while l>=0 and arr[l]<arr[l+1]:
                length+=1; l-=1
            while r<len(arr) and arr[r]<arr[r-1]:
                length+=1; r+=1
            longest=max(longest,length); i=r
        else:
            i+=1
    return longest
# Time: O(n), Space: O(1)


## Array of Products
Prefix and suffix products computed in two linear passes.

**Big-O:** see code cell comment.

In [ ]:

def array_of_products(arr: List[int]) -> List[int]:
    n=len(arr); res=[1]*n
    prefix=1
    for i in range(n):
        res[i]=prefix; prefix*=arr[i]
    suffix=1
    for i in range(n-1,-1,-1):
        res[i]*=suffix; suffix*=arr[i]
    return res
# Time: O(n), Space: O(1) extra


## Minimum Rewards
Sort by scores implicitly using left/right passes counting increasing streaks.

**Big-O:** see code cell comment.

In [ ]:

def min_rewards(scores: List[int]) -> int:
    n=len(scores); rewards=[1]*n
    for i in range(1,n):
        if scores[i]>scores[i-1]:
            rewards[i]=rewards[i-1]+1
    for i in range(n-2,-1,-1):
        if scores[i]>scores[i+1]:
            rewards[i]=max(rewards[i], rewards[i+1]+1)
    return sum(rewards)
# Time: O(n), Space: O(n)


## River Sizes (graph DFS)
Traverse the grid; DFS each unvisited land cell counting connected area.

**Big-O:** see code cell comment.

In [ ]:

def river_sizes(matrix: List[List[int]]) -> List[int]:
    if not matrix: return []
    visited=[[False]*len(matrix[0]) for _ in matrix]
    res=[]
    def dfs(r,c):
        stack=[(r,c)]; size=0
        while stack:
            x,y=stack.pop()
            if visited[x][y]: continue
            visited[x][y]=True
            if matrix[x][y]==0: continue
            size+=1
            for dx,dy in ((1,0),(-1,0),(0,1),(0,-1)):
                nx,ny=x+dx,y+dy
                if 0<=nx<len(matrix) and 0<=ny<len(matrix[0]) and not visited[nx][ny]:
                    stack.append((nx,ny))
        return size
    for i in range(len(matrix)):
        for j in range(len(matrix[0])):
            if not visited[i][j] and matrix[i][j]==1:
                res.append(dfs(i,j))
    return res
# Time: O(rc), Space: O(rc)


## Binary Trees: BFS/DFS
Level-order uses a queue; DFS uses recursion/stack. Each visits every node once.

**Big-O:** see code cell comment.

In [ ]:

class BTNode:
    def __init__(self,val,left=None,right=None):
        self.val=val; self.left=left; self.right=right
def bfs(root: BTNode) -> list[int]:
    if not root: return []
    q=[root]; out=[]
    while q:
        node=q.pop(0); out.append(node.val)
        if node.left: q.append(node.left)
        if node.right: q.append(node.right)
    return out
def dfs_preorder(root: BTNode) -> list[int]:
    res=[]
    def helper(node):
        if not node: return
        res.append(node.val)
        helper(node.left); helper(node.right)
    helper(root); return res
# Time: O(n), Space: O(n) worst-case


## BST Closest Value
Walk down the BST comparing distances; branch left/right by BST property.

**Big-O:** see code cell comment.

In [ ]:

class BST:
    def __init__(self, val):
        self.value=val; self.left=None; self.right=None
def find_closest_value(tree: BST, target: int) -> int:
    closest=tree.value
    node=tree
    while node:
        if abs(target-node.value)<abs(target-closest):
            closest=node.value
        node=node.left if target<node.value else node.right
    return closest
# Time: O(h), Space: O(1)


## Branch Sums & Node Depths
Recursive traversal accumulating path sums or depth counts.

**Big-O:** see code cell comment.

In [ ]:

def branch_sums(root: BTNode) -> list[int]:
    res=[]
    def helper(node, running):
        if not node: return
        running+=node.val
        if not node.left and not node.right:
            res.append(running); return
        helper(node.left, running); helper(node.right, running)
    helper(root,0); return res
def node_depths(root: BTNode, depth=0) -> int:
    if not root: return 0
    return depth + node_depths(root.left, depth+1) + node_depths(root.right, depth+1)
# Time: O(n), Space: O(h)


## Invert Binary Tree
Swap children recursively or iteratively; mirrors the tree.

**Big-O:** see code cell comment.

In [ ]:

def invert_tree(root: BTNode) -> BTNode:
    if not root: return root
    root.left, root.right = invert_tree(root.right), invert_tree(root.left)
    return root
# Time: O(n), Space: O(h)


## Min Height BST
Insert midpoints of sorted array to keep tree balanced via recursion.

**Big-O:** see code cell comment.

In [ ]:

def min_height_bst(sorted_array: List[int]) -> BST:
    def build(lo,hi):
        if lo>hi: return None
        mid=(lo+hi)//2
        node=BST(sorted_array[mid])
        node.left=build(lo, mid-1)
        node.right=build(mid+1, hi)
        return node
    return build(0, len(sorted_array)-1)
# Time: O(n), Space: O(n)


## Max Subset Sum No Adjacent
DP over array choosing max of take vs skip using rolling variables.

**Big-O:** see code cell comment.

In [ ]:

def max_subset_no_adjacent(arr: List[int]) -> int:
    if not arr: return 0
    if len(arr)==1: return arr[0]
    incl=arr[0]; excl=max(arr[0], arr[1])
    for n in arr[2:]:
        incl,excl=excl, max(excl, incl+n)
    return excl
# Time: O(n), Space: O(1)


## Number of Ways to Make Change
DP accumulating combinations for each coin and amount.

**Big-O:** see code cell comment.

In [ ]:

def ways_to_make_change(n: int, coins: List[int]) -> int:
    ways=[0]*(n+1)
    ways[0]=1
    for coin in coins:
        for amt in range(coin, n+1):
            ways[amt]+=ways[amt-coin]
    return ways[n]
# Time: O(n * c), Space: O(n)


## Levenshtein Distance
Classic DP grid comparing prefixes; fill row by row using edit costs.

**Big-O:** see code cell comment.

In [ ]:

def levenshtein(a: str, b: str) -> int:
    if len(a)<len(b): a,b=b,a
    prev=list(range(len(b)+1))
    for i,ca in enumerate(a,1):
        cur=[i]
        for j,cb in enumerate(b,1):
            if ca==cb:
                cur.append(prev[j-1])
            else:
                cur.append(1+min(prev[j-1], prev[j], cur[-1]))
        prev=cur
    return prev[-1]
# Time: O(len(a)*len(b)), Space: O(min(len(a),len(b)))


## Permutations & Powerset
Backtracking traverses decision tree; branching yields exponential outputs.

**Big-O:** see code cell comment.

In [ ]:

def permutations(arr: List[int]) -> List[List[int]]:
    res=[]
    def backtrack(path, used):
        if len(path)==len(arr):
            res.append(path[:]); return
        for i,n in enumerate(arr):
            if used[i]: continue
            used[i]=True; path.append(n)
            backtrack(path, used)
            path.pop(); used[i]=False
    backtrack([], [False]*len(arr)); return res
def powerset(arr: List[int]) -> List[List[int]]:
    res=[[]]
    for n in arr:
        res += [subset+[n] for subset in res]
    return res
# Time: O(n*n!), O(n*2^n); Space proportional to outputs


## Balanced Brackets
Push opening brackets onto stack; pop and validate matching closes.

**Big-O:** see code cell comment.

In [ ]:

def balanced_brackets(s: str) -> bool:
    pairs={')':'(',']':'[','}':'{'}; stack=[]
    for ch in s:
        if ch in '([{': stack.append(ch)
        elif ch in pairs:
            if not stack or stack.pop()!=pairs[ch]:
                return False
    return not stack
# Time: O(n), Space: O(n)


## Comprehensive Solution Catalog

Below are Python reference implementations for the **Easy** practice set requested. Each function is self-contained and includes inline Big-O notes. Medium and harder variants follow the same patterns documented earlier in the notebook (two-pointer arrays, heaps for k-way merges, DFS/BFS over graphs/trees, dynamic programming for subsequences, greedy for scheduling, and heap/union-find for graph spanning problems). Use these foundations to extend into the Medium/Hard/Very Hard lists.


In [ ]:
"""Easy practice set implementations with inline Big-O notes."""from collections import Counter, dequedef validate_subsequence(array, sequence):    it = iter(array)    return all(val in it for val in sequence)  # O(n) time, O(1) spacedef two_number_sum(array, target):    seen = set()    for x in array:        if target - x in seen:            return [target - x, x]        seen.add(x)    return []  # O(n) time, O(n) spacedef sorted_squared_array(arr):    res=[0]*len(arr)    l,r=0,len(arr)-1    for i in range(len(arr)-1,-1,-1):        if abs(arr[l])>abs(arr[r]):            res[i]=arr[l]**2;l+=1        else:            res[i]=arr[r]**2;r-=1    return res  # O(n) time, O(n) spacedef tournament_winner(competitions, results):    scores=Counter(); best=''    for (home,away),res in zip(competitions,results):        winner=home if res==1 else away        scores[winner]+=3        if scores[winner]>scores[best]:            best=winner    return best  # O(m) timedef non_constructible_change(coins):    coins.sort(); change=0    for coin in coins:        if coin>change+1:            return change+1        change+=coin    return change+1  # O(n log n)def transpose_matrix(matrix):    return [list(row) for row in zip(*matrix)]  # O(r*c)def find_closest_value_in_bst(tree, target):    node=tree; closest=tree.value    while node:        if abs(target-node.value)<abs(target-closest):            closest=node.value        node=node.left if target<node.value else node.right    return closest  # O(h)def branch_sums(root):    sums=[]    def dfs(node,total):        if not node:            return        total+=node.value        if not node.left and not node.right:            sums.append(total); return        dfs(node.left,total); dfs(node.right,total)    dfs(root,0); return sums  # O(n)def node_depths(root):    total=0; stack=[(root,0)]    while stack:        node,depth=stack.pop()        if not node: continue        total+=depth        stack.append((node.left,depth+1)); stack.append((node.right,depth+1))    return total  # O(n)def evaluate_expression_tree(tree):    if not tree.left and not tree.right:        return tree.value    left=evaluate_expression_tree(tree.left)    right=evaluate_expression_tree(tree.right)    if tree.value== -1: return left+right    if tree.value== -2: return left-right    if tree.value== -3: return left//right    if tree.value== -4: return left*right    raise ValueError('Unknown op')  # O(n)def depth_first_search(node, array):    array.append(node.name)    for child in node.children:        depth_first_search(child,array)    return array  # O(v+e)def minimum_waiting_time(queries):    queries.sort(); total=0; prefix=0    for q in queries[:-1]:        prefix+=q; total+=prefix    return total  # O(n log n)def class_photos(red, blue):    red.sort(); blue.sort(); first_red=red[0]<blue[0]    for r,b in zip(red,blue):        if (r>=b and first_red) or (b>=r and not first_red):            return False    return True  # O(n log n)def tandem_bicycle(red, blue, fastest):    red.sort(); blue.sort(reverse=fastest)    return sum(max(r,b) for r,b in zip(red,blue))  # O(n log n)def optimal_freelancing(jobs):    pay=[0]*7    for job in jobs:        deadline=min(6,job['deadline']-1)        for d in range(deadline,-1,-1):            if pay[d]==0:                pay[d]=job['payment']; break    return sum(pay)  # O(7*j)def remove_duplicates_from_linked_list(head):    cur=head    while cur and cur.next:        if cur.value==cur.next.value:            cur.next=cur.next.next        else:            cur=cur.next    return head  # O(n)def middle_node(head):    slow=fast=head    while fast and fast.next:        slow=slow.next; fast=fast.next.next    return slow  # O(n)def nth_fibonacci(n):    a,b=0,1    for _ in range(n):        a,b=b,a+b    return a  # O(n)def product_sum(array, depth=1):    total=0    for item in array:        if isinstance(item,list):            total+=product_sum(item,depth+1)        else:            total+=item    return total*depth  # O(n)def binary_search(array, target):    l,r=0,len(array)-1    while l<=r:        m=(l+r)//2        if array[m]==target:            return m        if array[m]<target: l=m+1        else: r=m-1    return -1  # O(log n)def find_three_largest_numbers(array):    top=[float('-inf')]*3    for x in array:        if x>=top[2]: top=[top[1],top[2],x]        elif x>=top[1]: top=[top[1],x,top[2]]        elif x>=top[0]: top=[x,top[1],top[2]]    return top  # O(n)def bubble_sort(arr):    a=arr[:]    n=len(a)    for i in range(n):        for j in range(0,n-i-1):            if a[j]>a[j+1]:                a[j],a[j+1]=a[j+1],a[j]    return a  # O(n^2)def insertion_sort(arr):    a=arr[:]    for i in range(1,len(a)):        key=a[i]; j=i-1        while j>=0 and a[j]>key:            a[j+1]=a[j]; j-=1        a[j+1]=key    return a  # O(n^2)def selection_sort(arr):    a=arr[:]    for i in range(len(a)):        m=i        for j in range(i+1,len(a)):            if a[j]<a[m]: m=j        a[i],a[m]=a[m],a[i]    return a  # O(n^2)def palindrome_check(s):    return s==s[::-1]  # O(n)def caesar_cipher_encryptor(s,key):    key%=26    return ''.join(chr(((ord(c)-97+key)%26)+97) for c in s)  # O(n)def run_length_encoding(s):    res=[];count=1    for i in range(1,len(s)+1):        if i<len(s) and s[i]==s[i-1] and count<9:            count+=1        else:            res.append(f"{count}{s[i-1]}"); count=1    return ''.join(res)  # O(n)def common_characters(strings):    common=Counter(strings[0])    for s in strings[1:]:        common&=Counter(s)    return sorted(list(common.elements()))  # O(n*k)def generate_document(characters, document):    counts=Counter(characters)    for c in document:        counts[c]-=1        if counts[c]<0:            return False    return True  # O(n)def first_non_repeating_character(string):    counts=Counter(string)    for i,ch in enumerate(string):        if counts[ch]==1:            return i    return -1  # O(n)def semordnilap(words):    seen=set(); pairs=[]    for w in words:        r=w[::-1]        if r in seen:            pairs.append([w,r])        seen.add(w)    return pairs  # O(n*k)